#### RESOURCES

In [ ]:
# SCRAPING
# https://brennan.io/2016/03/02/logging-in-with-requests/
# https://kazuar.github.io/scraping-tutorial/
    
# EXAMPLES OF MULTIPLE ACCOUNT LOG IN NOTIFICATIONS
# http://956sportsforum.com/forumdisplay.php?78-Multiple-Acount-detection
    
# GENERAL TECHNIQUES
# https://stackoverflow.com/questions/273192/how-can-i-create-a-directory-if-it-does-not-exist/14364249#14364249
# https://stackoverflow.com/questions/438894/how-do-i-stop-a-program-when-an-exception-is-raised-in-python
# https://stackoverflow.com/questions/13210436/get-full-traceback
# https://stackoverflow.com/questions/14058453/making-python-loggers-output-all-messages-to-stdout-in-addition-to-log
# https://stackoverflow.com/questions/40858658/python-logging-to-stdout-and-log-file

#### COOKIES

USES: vBulletin 3.7.4

There is a plug-in which detects multiple log-ins from the same computer.  I'm not entire sure
of the specifics but it accomplishes this by setting a cookie called IDstack.

http://www.vbulletin.org/forum/showthread.php?t=183268&page=27
    
The cookie can be deleted once logged in and it does not appear to return.  After logging out
and back-in again, the cookie will reappear.  Therefore, the program will need to check for and
delete this cookie upon each request.

IDstack:"%2C451297%2C"
    CreationTime:"Mon, 06 Mar 2017 20:25:56 GMT"
    Domain:"www.prisontalk.com"
    Expires:"Tue, 04 Jul 2017 20:27:27 GMT"
    HostOnly:true
    HttpOnly:false
    LastAccessed:"Mon, 06 Mar 2017 20:55:45 GMT"
    Path:"/"
    Secure:false
        
vBulletin also uses cookies to display when the site was last accessed, the last activity, 
last visit, etc.  These may need to be changed to avoid detection as well.

***NOTE: The following are *not* the key value pairs.  You must check for exact values.  If
you have cookie manager for firefox you can copy the raw values***

---
PHPSESSID
---
Host:                 www.prisontalk.com
Name:                 PHPSESSID
Path:                 %2F
Content:              uouvfmhmooh7tidfho4gsbaj64
Expires:              At end of session
Send for:             Any type of connection
Created:              March 6, 2017 at 4:13:47 PM
Last accessed:        March 6, 2017 at 4:15:05 PM
HTTP only:            No
This domain only:     No
Policy:               no information available
Status:               no information available
---

#### PRE LOGIN REQUEST HEADERS

In [ ]:
REQUEST HEADERS

Host: www.prisontalk.com
User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:58.0) Gecko/20100101 Firefox/58.0
Accept: text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8
Accept-Language: en-US,en;q=0.5
Accept-Encoding: gzip, deflate
Cookie: ap-settings-cached=N%7C2%7C5000%7Chttp%3A%2F%2Fwww.prisontalk.com%2Fptoads%7Caeroview%40gmail.com%7CThanks%2C%0D%0ADavid%0D%0Aaeroview%40gmail.com; __utma=160411260.263799196.1518124335.1518894718.1518899923.25; __utmz=160411260.1518124335.1.1.utmccn=(direct)|utmcsr=(direct)|utmcmd=(none); __qca=P0-1564290053-1518124335352; IDstack=%2C451297%2C%2C469607%2C; bblastvisit=1518127085; bblastactivity=0; bbsessionhash=c9d7625c482e698cdaf4e5d88d2fc6ac; PHPSESSID=ocd53ttttsp7fft2sbls12smv1; __utmc=160411260; bbpda=1; __utmb=160411260
Connection: keep-alive
Upgrade-Insecure-Requests: 1
Cache-Control: max-age=0

REPSONSE HEADERS 
Connection: keep-alive
Accept-Encoding: gzip, deflate
Accept: text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8
User-Agent: Mozilla/5.0 (Windows NT 6.1; rv:52.0) Gecko/20100101 Firefox/52.0
Accept-Language: en-US,en;q=0.5
DNT: 1
Host: www.prisontalk.com

RESPONSE COOKIES
bblastactivity: 0
bblastvisit: 1518982179
bbsessionhash: 2116591513c0a59d000602a392960a26

In [101]:
import json
import pandas as pd
import csv
from datetime import datetime
import collections
import re
import random
import time
import os
import logging
import sys
import errno
from pathlib2 import Path

from pprint import pprint

from bs4 import BeautifulSoup, NavigableString, Tag
import lxml.html
import requests
from string import capwords
from titlecase import titlecase

import smtplib

from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

print 'all done'

all done


In [102]:
firefox_win_ua = 'Mozilla/5.0 (Windows NT 6.1; rv:52.0) Gecko/20100101 Firefox/52.0'
firefox_mac_ua = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:58.0) Gecko/20100101 Firefox/58.0'
safari_mac_ua = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/604.4.7 (KHTML, like Gecko) Version/11.0.2 Safari/604.4.7'

domain = 'www.prisontalk.com'

headers =   {
                'Host': domain,
                'User-Agent': firefox_win_ua,
                'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
                'Accept-Language': 'en-US,en;q=0.5',
                'Accept-Encoding': 'gzip, deflate',
                'DNT': '1',
                'Connection': 'keep-alive'
            }

### THREAD SCRAPER

#### FOR FINDING GOOD TEST FORUMS OR REMOVING TEST FORUMS ALREADY RETRIEVED

In [59]:
# # FOR FINDING GOOD TEST FORUMS OR REMOVING TEST FORUMS ALREADY RETRIEVED

file_path = 'data/forums/'
log_path = file_path + 'logs/'
forums_csv = 'data/forum_ids.csv'
forum_list = get_forum_list(forums_csv)
test_forums = [forum_list[5], forum_list[6], forum_list[9]]

# # for idx, forum in enumerate(forum_list[:10]):
# #     print idx, forum['fid'], forum['threads'], forum['name']

# # for forum in test_forums:
# #     print forum['fid'], forum['threads'], forum['name']

#### TO REMOVE ALREADY RETRIEVED FORUMS WHEN RESUMING 

In [156]:
full_forum_list = get_forum_list(forums_csv)

path = 'data/forums/' # end in '/' so we can add filename later

retrieved_fids = os.listdir(path)#[1:] # skips .DS_Store IF THERE IS ONE
retrieved_fids.remove('logs')

remaining_forum_list = remove_retrieved_forums(full_forum_list, retrieved_fids)

# TO FIND NUMBER OF DIRECTORIES IT SHOULD EQUAL DO 'echo */ | wc'
# LOOK AT SECOND NUMBER, THEN SUBTRACT 1 ('logs')

print 'ORIGINAL FORUMS:', len(full_forum_list)
print 'RETRIEVED FORUMS:', len(retrieved_fids)
print 'FORUMS REMAINNIG:', len(remaining_forum_list)
print ''
print 'TOTAL FORUMS:', len(retrieved_fids) + len(remaining_forum_list)

ORIGINAL FORUMS: 43
RETRIEVED FORUMS: 25
FORUMS REMAINNIG: 18

TOTAL FORUMS: 43


#### DOES NOT REQURE LOGIN

In [157]:
# HUMANS START AT START PAGE (OOOOOOOOOO...HOW TRICKY OF ME...!)
s = requests.session()
response = s.get('http://prisontalk.com', headers=headers)

print response.status_code, response.url # 200 http://prisontalk.com/
# print_headers(response.headers)

time.sleep(3)

# THEN HUMANS CLICK ON THE FORUMS PAGE (AGAIN, A REAL TRICKSTER I AM.)
forum_list_url = 'http://www.prisontalk.com/forums/'
forum_list_res = s.get(forum_list_url, headers=headers)

print forum_list_res.status_code, forum_list_res.url # 200 http://prisontalk.com/forums/
# print_headers(response.headers)

200 http://prisontalk.com/
200 http://www.prisontalk.com/forums/


#### SETUP VARIABLES

In [158]:
# # THIS ANNOYS ME THAT I NEED TO REQUEST THE INITIAL PAGE TWICE 
# # BUT MY BRAIN IS TOO TIRED TO ABSTRACT OUT THE FIRST PART

# TODO: SEND AN UPDATE ONCE AN EVENING...?
# TODO: RENAME ALL TEST_FORUM(S) FORUM(S)

# MIN/MAX SCRAPE/BREAK TIMES IN MINUTES
min_time = 8
max_time = 18

# HOW LONG WE'LL INITIALLY SCRAPE FOR BEFORE TAKING A BREAK
start_time, scrape_time = get_next_scrape_times(min_time, max_time)

file_path = 'data/forums/'
log_path = file_path + 'logs/'

# LOAD ALL FORUMS
forums_csv = 'data/forum_ids.csv'
full_forum_list = get_forum_list(forums_csv)

# GET LIST OF ALREADY RETRIEVED ID'S FROM DIRECTORY AND REMOVE 'logs'
retrieved_fids = os.listdir(path)#[1:] # skips .DS_Store IF THERE IS ONE
retrieved_fids.remove('logs')

# GET REMAINING FORUMS BY REMOVING ALREADY RETRIEVED FORUMS FROM FULL LIST
test_forums = remove_retrieved_forums(full_forum_list, retrieved_fids)



# progress_states = ['forum_complete', 'downtime', 'continuing', 'finished'] # USED FOR TESTING
# forum_complete = False # I THINK THIS CAN BE MOVED INTO THE TRY CASE AND WE CAN REMOVE THE SWITCH IN THE THIRD IF...?\

#### SCRAPE

In [ ]:
# TODO: WHEN SMALLER FORUMS, SEE IF REMOVING SECOND 'FORUM COMPLETE' IS OKAY
# TODO: REMOVE FINAL WAIT TIME (AFTER LAST FORUM IS RETRIEVED)
# TODO: CHANGE 'CONTINUING' STATUS TO GIVE CORRECT FORUM IS STARTING A NEW ONE

# COULD DO WHILE LOOP BUT I HATE THE THOUGHT OF ACCIDENTALLY INDUCING INFINITY
for test_forum in test_forums:
    
    try:
        # STARTING NEW FORUM, (RE)SET FORUM COMPLETION TO FALSE
        forum_complete = False
        forum_id = test_forum['fid'] # NOT NECCESSARY BUT I USE IT LATER IN THE FOR LOOP
        
        test_forum['forum_url'] = get_forum_url(forum_id, 0) # GET INITIAL URL
        test_forum['num_pages'] = get_num_thread_pages(test_forum['forum_url'], headers) # GET NO. THREAD PAGES

        pages = range(test_forum['num_pages']) # TURN NUMBER OF THREAD PAGES INTO LIST
        
        logger = create_forum_logger(log_path, forum_id)
        logger.info('STARTING RETRIEVAL OF ' + str(test_forum['num_pages']) + ' PAGES OF FORUM ID ' + str(forum_id))

    # KEEPING TRY FOR NOW IN CASE WE NEED TO DEBUG, WILL NEED TO INDENT ALL BY ONE
    # try:

        for page in pages:
        # IF THERE ARE STILL PAGES TO RETRIEVE AND IT'S NOT BREAK TIME, ELSE, SEE ELSE

            # CREATE URL TO RETRIEVE
            thread_page_url = get_forum_url(forum_id, page)
            logger.info('GETTING: ' + str(thread_page_url.split('?')[-1])) #'f=39&order=DESC&sort=posts&pp=100&page=25'

            # GET PAGE
            thread_page = s.get(thread_page_url, headers=headers)
            logger.info('STATUS CODE: ' + str(thread_page.status_code))

            # IF YOU GET A GOOD STATUS CODE HOPEFULLY THE PAGE IS RETRIEVED THEN SAVE IT, IF NOT, RAISE EXCEPTION
            if not thread_page.raise_for_status():

                #SAVE FILE
                saved_file_location = save_html_file(thread_page, file_path, forum_id, page)
                logger.info('WROTE PAGE TO: ' + saved_file_location)

                # LET'S WAIT A FEW SECONDS SO WE DON'T SEEM SO MECHANICAL AND OVERLOAD THEY'RE SYSTEM
                page_wait_time = random.randint(6, 10)
                logger.info('WAITING: ' + str(page_wait_time) + ' SECONDS')

                # WAIT IN SECONDS BEFORE GETTING THE NEXT PAGE
                time.sleep(page_wait_time)

            # THE IMPLICATION HERE IS THAT SINCE IT'S NOT BREAK TIME WE'VE REACHED, TODO: PUT IN ELSE?
            if page == pages[-1]:

                logger.info('REACHED PAGE ' + str(page + 1) + ' OF ' + str(pages[-1] + 1) + ' FORUM ' + str(forum_id) + ' COMPLETE!')

                # SINCE WE'RE DONE WITH THIS THREAD, FLIP THE SWITCH TO UTILIZE NEXT 'IF'
                forum_complete = True

            if time_to_take_a_break(start_time, scrape_time) or forum_complete:

                # SET THE CURRENT STATUS TO CAN SEND THE CORRECT NOTIFICATION
                if forum_complete:
                    current_status = 'forum_complete'
                    forum_complete = False # RESET STATUS OF FORUM COMPLETION, MAY NOT NEED SINCE VAR IS RESET AT START?
                else:
                    current_status = 'downtime'

                # CALCULATE BREAK TIME IN MINUTES
                length_of_time = random.randint(min_time, max_time)
                logger.info('TAKING A ' + str(length_of_time) + ' MINUTE BREAK.  TALK SOON.')

                # LET ME KNOW CURRENT STATUS
                report_sent = send_progress_report(current_status, forum_id, length_of_time)

                # BREAK TIME! (IN SECONDS)
                time.sleep(length_of_time * 60)

                # CHANGE STATUS BACK TO RUNNING
                current_status = 'continuing'

                # LET ME KNOW THAT THINGS ARE RESUMING
                logger.info('WAIT TIME OVER. RESUMING PAGE RETRIEVAL\n')

                # CREATE NEW START TIME AND BREAK TIME            
                start_time, scrape_time = get_next_scrape_times(min_time, max_time)

                # LET ME KNOW WE'RE BACK ON THE CASE
                report_sent = send_progress_report(current_status, forum_id, scrape_time)

    # IF THERE IS ANY PROBLEM WHATSOEVER
    except Exception as err:

        logger.info('CRITICAL ERROR:', err.__class__.__name__, err)

        # PLEASE LET ME KNOW
        report_sent = send_critical_report(err, traceback)

        # LOG SUCCESS/FAILURE OF REPORT
        if report_sent:
            logger.info('WORD\'S OUT. REPORT SENT\n')
        else:
            logger.info('NOT SURE WHY BUT YOU COULD NOT BE REACHED BY YOURSELF.  BUMMER.\n')

        # SHUT IT DOWN. IF WE DON'T WE'LL KEEP RECEIVING NOTIFICATIONS
        print traceback.format_exc()

        raise
        
logger.info('WE DID IT. ALL FORUMS DOWNLOADED. PEACE.\n')
report_sent = send_progress_report('finished', forum_id, scrape_time)

#### PLACE TO TEST EXCEPTIONS

In [349]:
# for i in [0,1,2]:
    
#     try:
#         if i == 1:
#             1/0
#         else: 
#             print 'i is equal to', i
        
#     except Exception as err:

# #         print err.__class__.__name__, err
        
# #         print traceback.format_exc()

#         raise

### FUNCTIONS

#### SEND PROGRESS REPORT

In [87]:
def send_progress_report(progress, forum, minutes):
    
    try:

        logger.info('PROGRES REPORT! SENT YOU A TEXT/EMAIL')

        txt_msg = MIMEMultipart()

        txt_msg['From']= 'YOUR SCRAPER'
        txt_msg['To'] = '***REMOVED***'
        txt_msg['Subject'] = 'GOOD NEWS EVERYONE!'
        txt_body = 'CHECK YOUR EMAIL.'

        txt_msg.attach(MIMEText(txt_body))


        email_msg = MIMEMultipart()

        email_msg['From']=' YOUR SCRAPER'
        email_msg['To'] = '***REMOVED***'
        email_msg['Subject'] = 'PROGRESS REPORT'
        
        if progress == 'forum_complete':
        
            email_body =  'SO FAR SO GOOD. \r\n' \
                        + 'JUST FINISHED DOWNLOADING FORUM ' + str(forum) + '\r\n' \
                        + 'TAKING A BREAK FOR ' + str(minutes) + ' MINUTES TO KEEP UNDER THE RADAR\r\n' \
                        + 'I\'LL LET YOU KNOW WHEN I\'M BACK AT IT\r\n' \
                        + 'TAKE CARE!'
                    
        elif progress == 'downtime':
                
            email_body =  'SO FAR SO GOOD. \r\n' \
                        + 'STILL DOWNLOADING PAGES FROM FORUM ' + str(forum) + '\r\n' \
                        + 'GONNA TAKE SOME DOWNTIME FOR ' + str(minutes) + ' MINUTES TO KEEP UNDER THE RADAR' + '\r\n' \
                        + 'I\'LL LET YOU KNOW WHEN I\'M BACK AT IT\r\n' \
                        + 'BE SEEING YOU!'
                    
        elif progress == 'continuing':
            
            email_body =  'YAAAAAAAAWN. \r\n' \
                        + 'OKAY, GOING BACK AT IT FOR ANOTHER ' + str(minutes) + ' MINUTES\r\n' \
                        + 'CONTINUING TO DOWNLOAD PAGES FROM FORUM ' + str(forum) + '\r\n' \
                        + 'TALK SOON!'
                        
        elif progress == 'finished':
            
            email_body =  'ALL DONE WITH WHATEVER YOU WERE HAVING ME DO. \r\n' \
                        + 'MAYBE EXPAND THIS FUNCTION TO TELL YOU WHAT I\'VE FINISHED?  \r\n' \
                        + 'IN THE MEANTIME, I\'M PRETTY CACHED, AND GRATEFUL WE DIDN\'T GET BANNED \r\n' \
                        + 'PEACE OUT.'
        
        email_msg.attach(MIMEText(email_body, 'plain'))
        
        smtp_gmail = smtplib.SMTP('smtp.gmail.com', 587)
        smtp_gmail.starttls()
        smtp_gmail.login('***REMOVED***', '***REMOVED***')


#         smtp_gmail.sendmail(txt_msg['From'], txt_msg['To'], txt_msg.as_string())
        smtp_gmail.sendmail(email_msg['From'], email_msg['To'], email_msg.as_string())

        smtp_gmail.close()
        
        return True
        
    except Exception as err:
        
        err_type = err.__class__.__name__
        
        logger.info('COULDN\'T SEND PROGRESS REPORT:', err_type, err) 
        logger.info('TRIED TO REACH YOU. GOT VOICEMAIL. I THINK I\'LL JUST QUIT WHILE I\'M AHEAD')
        
        return False 

    

# progress_states = ['forum_complete', 'downtime', 'continuing', 'finished']
# forums = [11, 22, 33, 44 ]
# break_times = [12, 34, 56, 78]

# for progress_state, forum, break_time in zip(progress_states, forums, break_times):
    
#     report_sent = send_progress_report(progress_state, forum, break_time)

#     if report_sent:
#         print 'OMG.  FINALLY.'
#     elif not report_sent:
#         print 'IF THIS KEEPS HAPPENING, I\'LL WALK, I SWEAR.'

#### EMAIL SELF IF THERE IS AN ERROR SUCH THAT THE WHOLE THING HAS TO BE SHUT DOWN

In [88]:
def send_critical_report(err, tb):
    
    try:

        tb_str = tb.format_exc()
        
        # KEEPING THIS FOR NOW BUT NOT NECCESSARY ANYMORE B/C TRACEBACK CONTAINS ALL THE INFO
        err_type = err.__class__.__name__ 
        

        txt_msg = MIMEMultipart()

        txt_msg['From']=' YOUR SCRAPER'
        txt_msg['To'] = '***REMOVED***'
        txt_msg['Subject'] = ' DON\'T PANIC BUT...'
        txt_body = 'CHECK YOUR EMAIL.'

        txt_msg.attach(MIMEText(txt_body))


        email_msg = MIMEMultipart()

        email_msg['From']=' YOUR SCRAPER'
        email_msg['To'] = '***REMOVED***'
        email_msg['Subject'] = 'CRITICAL FAILURE'
        email_body = 'ERROR:\r\n' + \
                    '\r\n' + \
                    tb_str + \
                    '\r\n' + \
                    'TRIED TO REACH YOU.\r\n' + \
                    'GOT VOICEMAIL.\r\n' + \
                    'GET READY FOR THE THUNDERDOME.\r\n' + \
                    'SHUT IT DOWN FOLKS. SHUT IT DOWN.'

        email_msg.attach(MIMEText(email_body, 'plain'))


        smtp_gmail = smtplib.SMTP('smtp.gmail.com', 587)
        smtp_gmail.starttls()
        smtp_gmail.login('***REMOVED***', '***REMOVED***')

        
        logger.info('DOUBLE CRITICAL FAILURE. SENT YOU A TEXT/EMAIL')
        smtp_gmail.sendmail(txt_msg['From'], txt_msg['To'], txt_msg.as_string())
        smtp_gmail.sendmail(email_msg['From'], email_msg['To'], email_msg.as_string())

        smtp_gmail.close()
        
        return True
        
    except Exception as err:
        
        err_type = err.__class__.__name__
        
        logger.info('TRIPLE CRITICAL FAILURE: ' + err_type + ' ' + str(err)) 
        logger.info('TRIED TO REACH YOU. GOT VOICEMAIL. SHUT IT DOWN FOLKS! SHUT IT DOWN.')
        
        return False 


# TEST IN A LOOP BECAUSE PREVIOUSLY I DID NOT AND 
# IN A REAL WORLD SITUATION IN A LOOP, I WAS TEXT SPAMMED
# BY MY OWN PROGRAM

# file_path = 'data/forums/'
# log_name = 'forum_'+ str(forum_id) + '.log'
# msgfmt = '%(asctime)s %(levelname)s: %(message)s'
# datefmt= '%d/%m/%Y %I:%M:%S %p'

# reload(logging)
# logging.basicConfig(filename=file_path + log_name, level=logging.INFO, format=msgfmt, datefmt=datefmt)
# logger = logging.getLogger()

# for i in range(2):
    
#     try:
#         1/0
#     except Exception as err:

#         report_sent = send_critical_report(err, traceback)

#         if report_sent:
#             print 'OMG.  FINALLY.'
#         elif not report_sent:
#             print 'IF THIS KEEPS HAPPENING, I\'LL WALK, I SWEAR.'
#             raise
# #         sys.exit()
        

# # SHOULD PRINT THE FOLLOWING AND SEND *ONE* TEXT AND EMAIL OF MESSAGE ABOVE
# # OMG.  FINALLY.
# # An exception has occurred, use %tb to see the full traceback.
# # SystemExit

#### HELPERS

In [135]:
def remove_retrieved_forums(original_list, retrieved_fids):
    # USED TO BE int(forum['fid']) but now list of fids is strings
    return [forum for forum in original_list if forum['fid'] not in retrieved_fids]

# forums_csv = 'data/forum_ids.csv'
# original_forum_list = get_forum_list(forums_csv)
# retrieved_fids = [12, 1506, 38, 39, 44, 583, 584, 645, 72]

# new_forum_list = remove_retrieved_forums(original_forum_list, retrieved_fids)

# print 'TOTAL FORUMS:', len(original_forum_list)
# print 'RETRIEVED FORUMS:', len(retrieved_fids)
# print 'FORUMS REMAINNIG:', len(new_forum_list)

# # TOTAL FORUMS: 43
# # RETRIEVED FORUMS: 9
# # FORUMS REMAINNIG: 35

In [90]:
def save_html_file(thread_page, path, forum_id, page):
    
    # NEW DIRECTORY PATH, EX: 'data/forums/28/'
    forum_path = path + str(forum_id) + '/'
    # NEW FILENAME, EX: 'fid_39_pg_14.html 
    file_name = 'fid_' + str(forum_id) + '_pg_' + str(page + 1) + '.hmtl'
    
    # CREATE FORUM_PATH DIRECTORY IF IT DOESN'T EXIST, ELSE, MOVE ON
    Path(forum_path).mkdir(parents=False, exist_ok=True)

    #SAVE FILE
    with open(forum_path + file_name, 'w') as html_file:

        html_file.write(thread_page.content)
    
    return forum_path + file_name
    
# forum_list_url = 'http://www.prisontalk.com/forums/'
# forum_list_res = s.get(forum_list_url, headers=headers)

# print forum_list_res.status_code, forum_list_res.url
# save_html_file(forum_list_res, 'data/forums/', 12, 27)

In [91]:
def create_forum_logger(file_path, forum_id):

    log_file = 'forum_'+ str(forum_id) + '.log'
    msgfmt = '%(asctime)s %(levelname)s: %(message)s'
    datefmt= '%d/%m/%Y %I:%M:%S %p'
    
    reload(logging)
    logging.basicConfig(filename=file_path + log_file, level=logging.INFO, format=msgfmt, datefmt=datefmt)
    logging.Formatter.converter = time.gmtime # CHANGE TO UTC TIME
    logger = logging.getLogger()
    
    return logger 

# fid = 'fid'
# fpath = 'data/forums/'
# log_path = fpath + 'logs/'
# fpages = 'fpages'

# logger = create_forum_logger(log_path, fid)
# # SHOULD SEE?: DAY TIME LEVEL: STARTING RETRIEVAL OF fpages PAGES OF FORUM ID fid
# logger.info('STARTING RETRIEVAL OF ' + str(fpages) + ' PAGES OF FORUM ID ' + str(fid))

In [100]:
def get_next_scrape_times(time_one, time_two):

    start_time = datetime.now() # TO SUBTRACT LATER
    scrape_time = random.randint(time_one, time_two) # HOW LONG WE'LL SCRAPE FOR BEFORE TAKING A BREAK
    
    return start_time, scrape_time

# # get_next_scrape_times(90, 150)

# start, scrape = get_next_scrape_times(90, 150)
# print start, scrape

In [93]:
def time_to_take_a_break(start_time, minutes_to_scrape):
    
    current_time = datetime.now()
    # NOT REALLY NECCESSARY IN MINUTE CONDITIONS BUT TESTS ARE IN SECONDS AND I'M IMPATIENT
    seconds_to_scrape = int(minutes_to_scrape * 60.0) 
    
    elapsed_time = current_time - start_time
    
#     print elapsed_time.total_seconds(), seconds_to_scrape
    
    return elapsed_time.total_seconds() >= seconds_to_scrape

# min_to_scrape_false = 0.1341245 # ~8.047 seconds, returns False
# min_to_scrape_true = 0.08343145 # ~5.005 seconds, returns True

# start = datetime.now()
# time.sleep(5)
# time_to_take_a_break(start, min_to_scrape_false), time_to_take_a_break(start, min_to_scrape_true)

In [94]:
def get_forum_url(forum_id, page):

    forum = forum_id
    order = 'DESC'
    sort = 'posts'
    pages_per = '100'
    page = page + 1 # SINCE INEVITABLE ARRAY STARTS AT THE 0th ELEMENT, ADD 1

    forum_url = 'http://prisontalk.com/forums/forumdisplay.php?'+ \
                                                'f='+str(forum)+ \
                                                '&order='+order+ \
                                                '&sort='+sort+ \
                                                '&pp='+pages_per+ \
                                                '&page='+str(page)
    return forum_url

# 'http://prisontalk.com/forums/forumdisplay.php?f=30&order=DESC&sort=posts&pp=100&page=3'
# get_forum_url(30, 0)

In [95]:
def get_forum_list(csv_filename):
    
    forum_list = []
    
    with open(csv_filename) as csv_file:
        
        reader = csv.DictReader(csv_file)
        
        # # TODO: ADD GET RETRIEVAL TIME TO ORIGINAL DictWriter FUNCTION
        for forum in reader:
            
            forum['retrieval_time'] = get_retrieval_time(forum['threads'])
            
            forum_list.append(forum)
            
    return forum_list

# csv_filename = 'data/forum_ids.csv'
# forum_list = get_forum_list(csv_filename)

# test_forums = [forum_list[1], forum_list[8], forum_list[18]]

# for forum in test_forums[:3]:
    
#     print 'ID:', forum['fid'] # ID: 38
#     print 'NAME:', forum['name'] # NAME: Introduce Yourself!
#     print 'THREADS', forum['threads'] # THREADS 25870
#     print 'RETRIEVAL TIME:', get_retrieval_time(forum['threads']), 'minutes' # RETRIEVAL TIME: 30 minutes
#     print ''

In [96]:
# NOTE: THIS DOES NOT ASSUME EVERY THREAD PAGE WILL HAVE >100 THREADS
# NOT SURE WHY YOU SHOULD NOTE IT BUT I WROTE THAT A BIT AGO SO YOU'LL HAVE TO ASK ME THEN
def get_num_thread_pages(forum_url, headers):
    
    try:
        initial_forum_page = s.get(forum_url, headers=headers)
        
        thread_page = str(BeautifulSoup(initial_forum_page.content, 'html.parser'))
#         thread_page = str(thread_page)
        no_literals = thread_page.replace('\n', '').replace('\t', '').replace('\r', '')
        thread_page = BeautifulSoup(no_literals, 'html.parser')# skip u'html PUBLIC "-//W3C//
    
        # THERE IS A TOP AND BOTTOM NAV SO WE'LL JUST TAKE THE TOP
        nav = thread_page.find_all('div', {'class':'pagenav'})[0]
        page_x_of_y = nav.find_all('td', {'class':'vbmenu_control'})[0]
        num_pages = page_x_of_y.text.split(' ')[-1]
        
    except IndexError as er:
        
        num_pages = 1
    
    return int(num_pages)

# forum_id_39 = forum_list[0] # ASSUMING WE HAVEN'T SHUFFLED LIST
# forum_id_39['forum_url'] = get_forum_url(forum_id_39['fid'], 0)

# forum_id_39['num_pages'] = get_num_thread_pages(forum_id_39['forum_url'], headers)  #101 FOR FORUM 39
# print forum_id_39['num_pages'], 'PAGES OF 100 THREADS EACH:', forum_id_39['forum_url']
# # 101 PAGES OF 100 THREADS EACH: http://prisontalk.com/forums/forumdisplay.php?f=39&order=DESC&sort=posts&pp=100&page=1

In [97]:
def get_retrieval_time(threads):
    
    average_page_wait = 7
    
    pages = int(threads)/100
    
    total_seconds = pages * 7
    
    total_minutes = total_seconds/60
    
    return total_minutes

# test_forums = [forum_list[1], forum_list[8], forum_list[18]]

# prints
# ID: 38
# NAME: Introduce Yourself!
# THREADS 25870
# RETRIEVAL TIME: 30 minutes

# for forum in test_forums:
    
#     print 'ID:', forum['fid']
#     print 'NAME:', forum['name']
#     print 'THREADS', forum['threads']
#     print 'RETRIEVAL TIME:', get_retrieval_time(forum['threads']), 'minutes'
#     print ''

In [98]:
def print_headers(response):
    
    print 'RESPONSE HEADERS'
    for key, val in response.headers.iteritems():
        print key + ':', val
    print ''
    print 'REQUEST HEADERS'
    for key, val in response.request.headers.iteritems():
        print key + ':', val
        
# s = requests.session()
# test_response = s.get('http://prisontalk.com', headers=headers)
# print_headers(test_response)

# # RESPONSE HEADERS
# # Date: Mon, 26 Feb 2018 20:57:24 GMT
# # Server: Apache/2.2.3 (CentOS)
# # ...
# # Transfer-Encoding: chunked
# # Content-Type: text/html; charset=ISO-8859-1
    
# # REQUEST HEADERS
# # Connection: keep-alive
# # Accept-Encoding: gzip, deflate
# # ...
# # DNT: 1
# # Host: www.prisontalk.com

#### LOGIN

In [4]:
username = 'YOUR USERNAME'
password = 'YOUR PASSWORD'

login_form = {
                'vb_login_username': username, 
                'vb_login_password': password,
                'do': 'login',
                'url': '/forums/memberlist.php', 
                'vb_login_md5password': '',
                'vb_login_md5password_utf': '',
                's': '',
                'securitytoken': 'guest'
             }

# login_form

In [62]:
# HUMANS START AT START PAGE
s = requests.session()
response = s.get('http://prisontalk.com')

print response.status_code, response.url

# print 'RESPONSE HEADERS'
# for key, val in response.headers.iteritems():
#     print key + ':', val
# print ''
# print 'REQUEST HEADERS'
# for key, val in response.request.headers.iteritems():
#     print key + ':', val

200 http://prisontalk.com/


In [ ]:
prison_talk = BeautifulSoup(response.content, 'lxml')

In [10]:
# login_url = 'http://www.prisontalk.com/forums/login.php?do=login'

# login_response = s.post(login_url, data=login_form)
print login_response.status_code
login_page = BeautifulSoup(login_response.content, 'lxml')

In [17]:
# print login_page.prettify()

In [15]:
# CHECK THAT LOGIN SUCCEEDED
bool(login_page.find_all(string=re.compile('redirect')))

True

In [11]:
login_form = get_login_form(login_page, 'YOUR USERNAME', 'YOUR PASSWORD')
login_form
logged_in = s.post(login_url, data=login_form, headers=headers)
logged_in.status_code

200

In [12]:
def get_login_form(content, username, password):
    
    hidden_inputs = content.find_all('input', type='hidden')
    form = { hidden_input.get('name') : hidden_input.get('value') for hidden_input in hidden_inputs}
    form['vb_login_username'] = username
    form['vb_login_password'] = password
    
    return form

# login_form(login_page, '***REMOVED***', '***REMOVED***')

# SHOULD RETURN SOMETHING LIKE THIS
# {'childforums': '1',
#  'daysprune': '',
#  'do': 'login',
#  'exactname': '1',
#  'quicksearch': '1',
#  's': '9cb4afa5672281f960f38061b55124d4',
#  'securitytoken': 'guest',
#  'url': '/forums/memberlist.php?order=DESC&sort=posts&pp=100&page=1',
#  'vb_login_md5password': None,
#  'vb_login_md5password_utf': None,
#  'vb_login_password': '***REMOVED***',
#  'vb_login_username': '***REMOVED***'}
# login_form = login_form(login_page, '***REMOVED***', '***REMOVED***')
# logged_in = s.post(login_url, data=login_form, headers=headers)
# logged_in.status_code

#### LOGOUT

In [15]:
logout_form = get_logout_form(members_page)
logout_form
logged_out = s.post(login_url, data=logout_form, headers=headers)
logged_out.status_code

200

In [16]:
logout_page = BeautifulSoup(logged_out.content)
bool(logout_page.find_all(string=re.compile('All cookies cleared!')))

True

In [3]:
# actual_logout_url = 'http://www.prisontalk.com/forums/login.php?do=logout&logouthash=1488913889-27078f9aebdfa124592c6aec5bf7fb354d8ccc5a'
# actual_members_url = 'http://prisontalk.com/forums/memberlist.php?order=DESC&sort=posts&pp=100&page=1'

order = 'DESC'
sort = 'posts'
pages_per = '100'
page = 1

members_url = 'http://prisontalk.com/forums/memberlist.php?order='+order+'&sort='+sort+'&pp='+pages_per+'&page='+str(page)

# s = requests.session()
# response = s.get(members_url, headers=headers)
# print response.status_code

200


In [13]:
def get_logout_form(content):
    
    hidden_inputs = content.find_all('input', attrs={'type':'hidden'})
    logout_form = { hidden_input.get('name') : hidden_input.get('value') for hidden_input in hidden_inputs}
    logout_form['do'] = 'logout'
    logout_form['logouthash'] = logout_form['securitytoken']
    
    return logout_form

# logout_form(some_response)
# {'childforums': '1',
#  'daysprune': '',
#  'do': 'logout',
#  'exactname': '1',
#  'logouthash': '1490140816-e339d91901d66f6437b3b47164b82460ea981106',
#  'quicksearch': '1',
#  's': '',
#  'securitytoken': '1490140816-e339d91901d66f6437b3b47164b82460ea981106',
#  'usergroupid': '0'}

# logged_out = s.post(login_url, data=logout_form, headers=headers)
# logged_out.status_code

# lgo = BeautifulSoup(logged_out.content)
# bool(lgo.find_all(string=re.compile('All cookies cleared!')))

In [254]:
# TODO: I'm meh about this whole technique.  Something about it is not quite extensible...?
def get_logout_hash(href):
    return href and re.compile('logouthash').search(href)

#'http://www.prisontalk.com/forums/login.php?do=logout&logouthash=1488913889-27078f9aebdfa124592c6aec5bf7fb354d8ccc5a'
# ml = BeautifulSoup(members_list.text)
# get_logout_url(ml)

In [351]:
page = BeautifulSoup(logged_out.content)

with open('logged_out'+str(2)+'.html', 'w') as html_file:
    html_file.write(page.prettify('utf-8'))

#### NOT SURE WHY I FEEL I NEED TO KEEP THIS FOR NOW

In [360]:
from datetime import timedelta

# type(elapsed_time.total_seconds())

start_time = datetime.now()
scrape_time = timedelta(seconds=5)

print 'START:', start_time, 'SCRAPE:', scrape_time, 'START + SCRAPE:', start_time + scrape_time

START: 2018-02-22 06:39:34.675765 SCRAPE: 0:00:05 START + SCRAPE: 2018-02-22 06:39:39.675765


In [291]:
# # THIS ENDED UP BEING STUPID BUT IT TOOK ME A LONG TIME TO FIGUIRE OUT SO
# # I'M LEAVING IT FOREVER AS A TESTIMENT TO MY HATRED OF REGEX
# no_carraige = members[0].text.replace('\n', '').strip()
# fields = re.split('\s{2,}', no_carraige)
# fields

In [168]:
# # TURNED INTO get_num_thread_pages() KEEPING FOR NOW JUST IN CASE
# # TEST LIST IS FORUM 39, FOR NOW, START AT FIRST PAGE WHICH WILL BE 0th ELEMENT, MAYBE RANDOMIZE LATER
# test_forum['forum_url'] = get_forum_url(test_forum['fid'], 0) 

# initial_forum_page = s.get(test_forum['forum_url'], headers=headers)

# thread_page = BeautifulSoup(initial_forum_page.content, 'html.parser')
# thread_page = str(thread_page)
# no_literals = thread_page.replace('\n', '').replace('\t', '').replace('\r', '')
# thread_page = BeautifulSoup(no_literals, 'html.parser')# skip u'html PUBLIC "-//W3C//

# test_forum['num_pages'] = get_num_thread_pages(thread_page)
# print test_forum['num_pages'], 'PAGES OF 100 THREADS EACH:', test_forum['forum_url']
# # 101 PAGES OF 100 THREADS EACH: http://prisontalk.com/forums/forumdisplay.php?f=39&order=DESC&sort=posts&pp=100&page=1

101 PAGES OF 100 THREADS EACH: http://prisontalk.com/forums/forumdisplay.php?f=39&order=DESC&sort=posts&pp=100&page=1


In [166]:
# NOTE: THIS DOES NOT ASSUME EVERY THREAD PAGE WILL HAVE >100 THREADS

# def get_num_thread_pages(el):
    
#     try:
    
#         # THERE IS A TOP AND BOTTOM NAV SO WE'LL JUST TAKE THE TOP
#         nav = el.find_all('div', {'class':'pagenav'})[0]
#         page_x_of_y = nav.find_all('td', {'class':'vbmenu_control'})[0]
#         num_pages = page_x_of_y.text.split(' ')[-1]
        
#     except IndexError as er:
        
#         num_pages = 1
    
#     return int(num_pages)

# # get_num_thread_pages(thread_page)  #101 FOR FORUM 39